In [2]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark import SparkFiles

In [3]:
spark = SparkSession.builder \
    .appName("NYC-Taxi-Tripdata-2024") \
    .config("spark.driver.memory", "4g") \
    .master("local") \
    .getOrCreate()

print(f'The current spark version is {spark.version}')

The current spark version is 3.5.0


In [ ]:
# ------------ Yellow NYC Taxi Trip ------------ #

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [ ]:
# file_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet'
df_yellow_dataset = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [ ]:
#2 question

In [ ]:
df_yellow_dataset \
    .repartition(4) \
    .write \
    .parquet('data/pq/parition/yellow/4/', mode='overwrite')

In [ ]:
#3 question

In [ ]:
df_pq \
    .withColumn('pickup_date', F.to_date(df_pq.tpep_pickup_datetime)) \
    .withColumn('dropoff_datetime', F.to_date(df_pq.tpep_dropoff_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .count()

In [ ]:
#4 question

In [ ]:
df_test = df_pq.withColumn("trip_duration_hours", 
                   (F.to_timestamp((unix_timestamp("tpep_dropoff_datetime"))) - F.to_timestamp((unix_timestamp("tpep_pickup_datetime")))))

# Find the longest trip duration
max_duration = df_test.agg(max("trip_duration_hours")).collect()[0][0]

print(f"Longest trip duration in hours: {max_duration}")

In [ ]:
#6 question

In [4]:
# ------------ NYC Taxi Zone ------------ #

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [ ]:
# Write csv file as parquet file
df_taxi_zone_lookup = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)

In [ ]:
# Save as parquet file
df_taxi_zone_lookup.write.mode("overwrite").parquet("parquet/taxi_zone_lookup")

In [ ]:
# Show the schema and sample data
print("Schema:")
df_taxi_zone_lookup.printSchema()

In [ ]:
df_pq.createOrReplaceTempView('yellow_tripdata')
df_taxi_zone_lookup.createOrReplaceTempView('zone')

In [ ]:
least_pickup_id = spark.sql("""
SELECT PULocationID, number FROM (
    SELECT 
        PULocationID,
        COUNT(PULocationID) as number
    FROM
        yellow_tripdata
    GROUP BY
        PULocationID
) t
ORDER BY number ASC
LIMIT 1
""")

In [ ]:
least_pickup_zone.createOrReplaceTempView('least_pickup_zone')

In [ ]:
spark.sql("""
SELECT 
    LocationID,
    Zone
FROM
    zone z 
JOIN least_pickup_zone lpu
ON z.LocationID = lpu.PULocationID
""").show()